<a href="https://colab.research.google.com/github/fadh1l/DR-pred-IDRiD/blob/main/DR_severity_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
import pandas as pd
import cv2
import os
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/MyDrive/Colab Notebooks/IDRiD/archive.zip"
clear_output()

In [ ]:
details = pd.read_csv('/content/Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv')

In [ ]:
details.head()

,Image name,Retinopathy grade,Risk of macular edema,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,IDRiD_001,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IDRiD_002,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IDRiD_003,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IDRiD_004,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IDRiD_005,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
details = details.iloc[:, :3]

In [ ]:
details['Retinopathy grade'].describe()

count    413.000000
mean       1.719128
std        1.387723
min        0.000000
25%        0.000000
50%        2.000000
75%        3.000000
max        4.000000
Name: Retinopathy grade, dtype: float64

In [ ]:
image_path = '/content/Disease Grading'
img = cv2.imread(image_path)

In [ ]:
os.mkdir('/content/Renamed Images')

In [ ]:
# !rm -rf ./Renamed\ Images

In [ ]:
csv_path = "/content/Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv"
image_folder = "/content/Disease Grading/1. Original Images/a. Training Set"
output_folder ='/content/Renamed Images'
df = pd.read_csv(csv_path)
for idx, row in df.iterrows():
  og_img_name = f"{row['Image name']}.jpg"
  sev = row['Retinopathy grade']
  new_img_name = f'img{idx}_{sev}.jpg'
  og_img_path = os.path.join(image_folder,og_img_name)
  new_img_path = os.path.join(output_folder, new_img_name)
  shutil.copy(og_img_path, new_img_path)

In [ ]:
!pip install torch torchvision timm
clear_output()

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch

class FundusDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.image_files = [f for f in os.listdir(data_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_name = self.image_files[idx]
        image_path = os.path.join(self.data_folder, image_name)
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        severity = int(image_name.split('_')[-1][0])

        return image, severity

# Specify the path to your data folder
data_folder = output_folder

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create datasets and dataloaders
train_dataset = FundusDataset(data_folder, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import torch
import timm

# Load Swin Transformer model
model = timm.create_model("swin_base_patch4_window7_224", pretrained=True)

# Modify the classifier head for regression (severity is a regression task)
model.head = torch.nn.Linear(model.head.in_features, 1)


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.float().to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 7, 7, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (7) must match the size of tensor b (32) at non-singleton dimension 2